In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Store all numbers of stores in a list
store_restaurant = pd.read_excel('store_restaurant.xlsx')
store_number  = store_restaurant.STORE_NUMBER
print(store_number)

## Match the order, keep the Recipe Id

In [ ]:
menuitem = pd.read_csv('menuitem.csv') 
menu_items = pd.read_csv('menu_items.csv') 
menu_items = menu_items.rename(columns={'MenuItemId': 'Id'})
orders = pd.merge(menuitem, menu_items, how="left", on=["PLU", "Id"]).drop(columns=['PLU','Id'])
orders['date'] = pd.to_datetime(orders['date'])
orders = orders.rename(columns={'Quantity':'Quantity_orders'})
orders

## Match the main ingredient and the quantity

In [ ]:
recipe_ingredient_assignments = pd.read_csv('recipe_ingredient_assignments.csv')
orders_with_ingredient = pd.merge(orders, recipe_ingredient_assignments, how="left", on=["RecipeId"])
orders_with_ingredient['Quantity'] = orders_with_ingredient['Quantity'] * orders_with_ingredient['Quantity_orders']
orders_with_ingredient = orders_with_ingredient.drop(columns=['Quantity_orders'])
orders_with_ingredient

## Match the sub ingredient and the quantity

In [ ]:
recipe_sub_recipe_assignments = pd.read_csv('recipe_sub_recipe_assignments.csv')
orders_with_sub_ingredient = pd.merge(orders, recipe_sub_recipe_assignments, how="left", on=["RecipeId"])
orders_with_sub_ingredient['Factor'] = orders_with_sub_ingredient['Factor'] * orders_with_sub_ingredient['Quantity_orders']
orders_with_sub_ingredient = orders_with_sub_ingredient.drop(columns=['Quantity_orders'])
orders_with_sub_ingredient = orders_with_sub_ingredient.dropna()
orders_with_sub_ingredient

In [ ]:
sub_recipe_ingr_assignments = pd.read_csv('sub_recipe_ingr_assignments.csv')
orders_with_sub_ingredient_1 = pd.merge(orders_with_sub_ingredient, sub_recipe_ingr_assignments, how="left", on=["SubRecipeId"])
orders_with_sub_ingredient_1

In [ ]:
orders_with_sub_ingredient_2 = orders_with_sub_ingredient_1.copy()
orders_with_sub_ingredient_2['Quantity(factor)'] = orders_with_sub_ingredient_2.Factor * orders_with_sub_ingredient_2.Quantity
orders_with_sub_ingredient_2

In [ ]:
orders_with_sub_ingredient_3 = orders_with_sub_ingredient_2.copy()
orders_with_sub_ingredient_3=orders_with_sub_ingredient_3.drop(columns=['Factor','Quantity','SubRecipeId'])
orders_with_sub_ingredient_3 = orders_with_sub_ingredient_3.rename(columns={'Quantity(factor)':'Quantity'})
orders_with_sub_ingredient_3

## Append the main ingredient and sub ingredient

In [ ]:
orders_with_all_ingredient = pd.concat([orders_with_sub_ingredient_3,orders_with_ingredient])
orders_with_all_ingredient = orders_with_all_ingredient[orders_with_all_ingredient.Quantity != 0]
orders_with_all_ingredient = orders_with_all_ingredient.drop(columns=['RecipeId'])
orders_with_all_ingredient

## Keeping only lettuce

In [ ]:
orders_with_lettuces = orders_with_all_ingredient[orders_with_all_ingredient.IngredientId == 27]
orders_with_lettuces

## Calculate the demand of lettuce

In [ ]:
lettuce_demand = pd.DataFrame(data=orders_with_lettuces.groupby(by=['StoreNumber','date'])['Quantity'].sum())
lettuce_demand = lettuce_demand.reset_index()
lettuce_demand

In [ ]:
store_number_0 = store_number[0]
store_number_1 = store_number[1]
store_number_2 = store_number[2]
store_number_3 = store_number[3]

In [ ]:
lettuce_demand_46673 = lettuce_demand[lettuce_demand.StoreNumber == store_number_0].sort_values(by=['date'])

lettuce_demand_4904 = lettuce_demand[lettuce_demand.StoreNumber == store_number_1].sort_values(by=['date'])

lettuce_demand_12631 = lettuce_demand[lettuce_demand.StoreNumber == store_number_2].sort_values(by=['date'])

lettuce_demand_20974 = lettuce_demand[lettuce_demand.StoreNumber == store_number_3].sort_values(by=['date'])

lettuce_demand_46673.to_csv('lettuce_demand_46673.csv')
lettuce_demand_4904.to_csv('lettuce_demand_4904.csv')
lettuce_demand_12631.to_csv('lettuce_demand_12631.csv')
lettuce_demand_20974.to_csv('lettuce_demand_20974.csv')